In [ ]:
import pandas as pd
import numpy as np
#from pandas import ExcelWriter
from math import floor
import re

In [ ]:
#df = pd.read_csv('Outputs/now_v2.csv')
df = pd.read_excel('TTOP 06 Jun Up2.xlsx')   #WAO Daily
df = df[((df['Impact']!= 'LEVEL4') & (df['Impact']!= 'MINOR'))]

In [1]:
pat_name = '\\A[A-Z]{2}[1-9]*[GLU]{0,3}[0-9]{4}'; #'[A-F]{1}'
df['CellName'] = df['CellName'].astype(str);
df['CellName'] = df['CellName'].str.upper();
df['Sector'] = df['CellName'].apply(lambda x:  ''.join(re.findall(pat_name, x)));
df = df[df['Sector'] != ''];
df['Site']=np.where(len(df['Sector']) > 7, df['Sector'].str[:8], df['Sector'].str[:6]);
df.drop(columns = ['Sector'], inplace = True)

NameError: name 'df' is not defined

In [ ]:
df['StartOfOutage'] = pd.to_datetime(df['StartOfOutage'])
df['EndOfOutage'] = pd.to_datetime(df['EndOfOutage'])
df.loc[df['Tech']=='4G','NE'] = df.loc[df['Tech']=='4G','NE'] + '_LTE'

In [ ]:
df.rename(columns={'Duration': 'Down_Time(Seconds)', 'NE': 'BSC-RNC'}, inplace = True)
df.sort_values(by = ['BSC-RNC','StartOfOutage','Down_Time(Seconds)'], inplace = True)

In [ ]:
#print(type(df['StartOfOutage'][0]))
df2 = df[['StartOfOutage','Down_Time(Seconds)']].diff()#.dt.total_seconds()/3600
df2.columns = ['d_st_hour','d_duration']

In [ ]:
df['diff'] = (df['BSC-RNC'] != df['BSC-RNC'].shift(1).fillna(df['BSC-RNC'])).astype(int)

In [ ]:
df = pd.concat([df, df2], axis = 1)

In [ ]:
df['d_st_hour'] = df['d_st_hour'].dt.total_seconds()/3600
df['d_duration'] = df['d_duration']/3600

In [ ]:
df['change_flag'] = 0
df[['d_st_hour','d_duration']] = df[['d_st_hour','d_duration']].fillna(0)

In [ ]:
flag_old_grouping = False
if flag_old_grouping:
    df.loc[(df['d_st_hour'] != 0) | (df['d_duration'].abs()> np.minimum(1/6, 0.1 * df['Down_Time(Seconds)']/3600)),'change_flag'] = 1
else: # new grouping
    df.loc[(df['d_st_hour'] != 0) | (df['d_duration'].abs()> \
                                     np.minimum(1/3, 
                                                np.maximum(0.1,(4/75 + 7/25* df['Down_Time(Seconds)']/3600)) * df['Down_Time(Seconds)']/3600)),'change_flag'] = 1


In [ ]:
#df_all['cs'] = df_all['change_flag'].cumsum()
#df_all
df['no'] = df.groupby(['BSC-RNC'])['change_flag'].cumsum()

In [ ]:
#ddf = df[['BSC-RNC','StartOfOutage','Down_Time(Seconds)','diff','d_st_hour','d_duration','change_flag','no']]
#ddf.to_csv('Outputs/wao_now.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
#print(df.head())
df2 = df.copy()
tdf = df.groupby(['BSC-RNC','no'])['Site'].nunique()#transform('nunique')

In [ ]:
tdf = tdf.reset_index().reset_index()

In [ ]:
tdf.rename(columns={'Site': 'NumSitePerGroup','index':'GroupNumber'}, inplace = True)

In [ ]:
#print(tdf.head())
#tdf.to_csv('wao_22.csv', index = False, encoding = 'utf-8-sig')
# merge on no and BSC-RNC
df = df.merge(tdf, left_on = ['BSC-RNC','no'], right_on = ['BSC-RNC','no'])

In [ ]:
SITE_COUNT_BSC = pd.read_excel('../Create Ticket/Grouping/SITE COUNT (BSC-RNC-City-Province)_2G3G4G.xlsx',sheet_name='BSC')
SITE_COUNT_RNC = pd.read_excel('../Create Ticket/Grouping/SITE COUNT (BSC-RNC-City-Province)_2G3G4G.xlsx',sheet_name='RNC')
SITE_COUNT_EqRNC = pd.read_excel('../Create Ticket/Grouping/SITE COUNT (BSC-RNC-City-Province)_2G3G4G.xlsx',sheet_name='Equivalent RNC')
SITE_COUNT_EqRNC['ElementID'] = SITE_COUNT_EqRNC['ElementID'] + '_LTE'
SITE_COUNT_NE = pd.concat([SITE_COUNT_BSC, SITE_COUNT_RNC, SITE_COUNT_EqRNC], axis = 0, sort = True)
SITE_COUNT_NE.rename(columns={'ElementID': 'BSC-RNC'}, inplace = True)
SITE_COUNT_NE['SITE_COUNT(30%)'] = SITE_COUNT_NE['SITE_COUNT']*0.3

In [ ]:
print('merging ...')
df = df.merge(SITE_COUNT_NE, left_on = ['BSC-RNC'], right_on = ['BSC-RNC'], how = 'left')


In [ ]:
df['outage_percent'] = df['NumSitePerGroup']/df['SITE_COUNT']

In [ ]:
df.to_excel(r'Grouping/Outage_Grouping_NE_06Jun2.xlsx', index = False, encoding = 'utf-8-sig')